#### Loading training database

In [1]:
import requests, json, os, warnings, re
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import sys
import glob
data_raw= pd.read_json("./1.TrainingData/all.json") 

In [2]:
data_raw[0:1]


Exclude appId                                            comment  \
0     NaN  None  Besides the occasional crash, this is an amazi...   

                       dataSource  date   fee  future   id label  \
0  RE2014_app_and_play_store_apps  None  None       0  264   Bug   

                                  lemmatized_comment  ...  reviewId  reviewer  \
0  besides the occasional crash, this be an amaze...  ...     10946      None   

   sentiScore  sentiScore_neg  sentiScore_pos  \
0           3              -1               3   

                                             stemmed  \
0  besid the occa crash, thi is an amaz produc wi...   

                                   stopwords_removal  \
0  besides occasional crash, this amazing product...   

                     stopwords_removal_lemmatization  \
0  besides occasional crash, this amaze product w...   

                              stopwords_removal_nltk           title  
0  besides occasional crash, amazing product tons...  Almost perfect  

[1 rows x 25 columns]

In [3]:
traindata_noRating=data_raw[data_raw["label"]!="Rating"]
traindata_Rating=data_raw[data_raw["label"]=="Rating"]
traindata_Rating_sample=traindata_Rating.sample(n=1000, random_state=1)

In [4]:
trainData = pd.concat([traindata_noRating, traindata_Rating_sample], axis=0)
trainData.label.value_counts()

Rating            1000
UserExperience     607
Bug                370
Feature            252
Name: label, dtype: int64

#### 1.  STARTING TO TRAIN THE MODEL

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

def measuretool(df,classvalue):   
    df["validate"]=df["response"]==df["predict"]
    a_value=sum(df[df["predict"]==classvalue]["validate"])
    ac_value=len(df[df["predict"]==classvalue])
    ab_value=len(df[df["response"]==classvalue])
    Precision_value=a_value/ac_value
    Recall_value=a_value/ab_value
    F1_value=2*Precision_value*Recall_value/(Precision_value+Recall_value)
    return pd.DataFrame({'Precision':[Precision_value], 'Recall':[Recall_value], 'F1_value' :[F1_value]})
        


#### 1.1. BOW + Bigram - Stopwords_Removal + Lemmatization

In [7]:
# Create training predictors
trainText=trainData.stopwords_removal_lemmatization
vect_11 = CountVectorizer(ngram_range=(2, 2))
X_train = vect_11.fit_transform(trainText).toarray()
Y_train = trainData.label

In [8]:
#Training model
classifier_11 = GaussianNB()
classifier_11.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [9]:
# Get predict value on the training dataset
Y_train_pred_11 = classifier_11.predict(X_train)
# Combince result
df_11 = pd.DataFrame({'response':Y_train, 'predict':Y_train_pred_11})
print(measuretool(df_11,"Bug"))
print(measuretool(df_11,"UserExperience"))
print(measuretool(df_11,"Feature"))
print(measuretool(df_11,"Rating"))

   Precision    Recall  F1_value
0   0.972452  0.954054  0.963165
   Precision   Recall  F1_value
0        1.0  0.76771  0.868593
   Precision  Recall  F1_value
0   0.581986     1.0  0.735766
   Precision  Recall  F1_value
0   0.881075   0.852  0.866294


#### 2. STARTING TESTING MODEL

#### 2.1 Loading Testing Data 

In [24]:
Testingdata= pd.read_csv("./2.TestingData/Testing300Sample_completed.csv") 
len(Testingdata)

300

#### 2.1 Preprocessing the testing data

In [25]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

# joing string
def join_func(x):
    x = " ".join(x)
    return x

#Preprocessing the data: Remove none alphabetic characters/Make the word lower case
def cleantext(x):       
    x = re.sub('[^A-Za-z]', ' ', x)
    x = str(x).lower()
    return x

#Remove the stop words
def stopword(x):
    x= word_tokenize(x)
    for word in x:
        if word in stopwords.words('english'):
            x.remove(word)
    return join_func(x)
    
#stemming
stemmer = PorterStemmer()
def stem_fuc(x):
    x= word_tokenize(x)
    for i in range(len(x)):
        x[i] = stemmer.stem(x[i])
    return join_func(x)

In [26]:
# Clean Test Data
X_testing=Testingdata.text
X_testing=X_testing.apply(cleantext)


In [27]:
# stop_word
X_testing=X_testing.apply(stopword)

In [28]:
#stemming
X_testing=X_testing.apply(stem_fuc)


#### 2.2 BOW + Bigram - Stopwords_Removal + Lemmatization (testing)

In [29]:
# Vecotorizer BOW + Bigram for testing data
vect11_testing = CountVectorizer(ngram_range=(2, 2),vocabulary=vect_11.vocabulary_)
X_testing_matrix = vect11_testing.fit_transform(X_testing).toarray()
Y_testing=Testingdata.label



In [30]:
# Get predict value on the training dataset with BOW +Bigram model
Y_testing_pred_11 = classifier_11.predict(X_testing_matrix)
df_11_testing = pd.DataFrame({'response':Y_testing, 'predict':Y_testing_pred_11})


In [31]:
print(measuretool(df_11_testing,"Bug"))
print(measuretool(df_11_testing,"UserExperience"))
print(measuretool(df_11_testing,"Feature"))
print(measuretool(df_11_testing,"Rating"))

   Precision    Recall  F1_value
0   0.296296  0.190476  0.231884
   Precision   Recall  F1_value
0   0.535714  0.16129  0.247934
   Precision    Recall  F1_value
0   0.111702  0.567568  0.186667
   Precision    Recall  F1_value
0   0.368421  0.164062  0.227027
